In [9]:
import pandas as pd
from utils import transformers_bert_completions, load_splits, load_models
from utils_model_sampling import sample_across_models, beta_utils

from yyy_analysis import examples_figure

ImportError: cannot import name 'examples_figures' from 'yyy_analysis' (/home/nwong/chompsky/childes/child_listening_continuation/child-directed-listening/yyy_analysis/__init__.py)

In [ ]:
# Changed this notebook to use the +/- 20 context newly generated versions.

# Written to match load_models.query_model_title
default_args = {
    'split' : 'all',
    'dataset' : 'all', 
    'is_tags' : False,
    'context_num' : 20,
}

childes_all_title = load_models.query_model_title(model_type = 'childes', **default_args)
adult_all_title = load_models.query_model_title(model_type = 'adult', **default_args)
unigram_title = load_models.query_model_title(model_type = 'data_unigram', **default_args)

In [2]:
eval_data = load_splits.load_eval_data_all('all', 'all')

# Is it possible that success_utts already has the glosses? Need to check this

success_utts = eval_data['success_utts']
yyy_utts = eval_data['yyy_utts']
all_tokens_phono = eval_data['phono']

initial_vocab, cmu_in_initial_vocab = load_models.get_initial_vocab_info()
utt_glosses = pd.concat([success_utts, yyy_utts])

# Technically only reconstructs the success utts and yyy utts,
# not the xxx or other utts,
# but will try to see if it's enough to visualize the specific data points requested in this notebook.

Change the tokenizer from model output2 to be dynamic if possible


In [3]:
success_utts_with_gloss = success_utts.copy() # Already merged with gloss in new Providence notebook
success_utts_with_gloss['num_tokens'] = [len(x.split(' ')) for x in success_utts_with_gloss['gloss']]


' | '.join(success_utts_with_gloss.loc[success_utts_with_gloss.num_tokens ==4 ].gloss[0:100])

"pop pop pop pop | another one another one | I don't want to | apple apple apple apple | a red a pink | oh A B c | Mom what is that | a green a blue | beep beep beep beep | a b a b | one two three four | did you did you | one two one two | one two one two | b e a o | what is this two | no no no no | a cars a cars | a two a two | beep beep beep beep | one two one one | a tree a tree | way up over there | Mom Mommy Mom Mommy | where's Daddy where's Dada | hi Gigi hi Gigi | a ball a ball | a four a four | another one another one | mah mah mah mah | what's that what's this | what's that a ball | five five five five | no no Mommy no | mah muh mm mm | one one two three | a green a green | a green a green | a square a square | I did a square | another one another one | no no no no | no no no no | another one another one | a ball a ball | a ball a ball | red circle what's that | a ball a ball | a four a five | Mommy I did it | look at the green | Mommy can you fix | I try to push | Mommy help 

In [4]:
selected = success_utts_with_gloss.loc[success_utts_with_gloss.gloss=='I want to read'].utterance_id 

In [5]:
all_tokens_phono.columns

Index(['token', 'id', 'gloss', 'transcript_id', 'utterance_order',
       'speaker_code', 'target_child_name', 'target_child_age', 'type',
       'punct', 'speaker_code_simple', 'gloss_with_punct', 'token_id',
       'seq_utt_id', 'actual_phonology', 'model_phonology', 'bert_token_id',
       'model_phonology_clean', 'actual_phonology_clean',
       'model_phonology_no_dia', 'actual_phonology_no_dia', 'cv_raw',
       'cv_collapsed', 'num_vowels', 'in_vocab', 'success_token', 'yyy_token',
       'partition'],
      dtype='object')

In [6]:
all_tokens_phono.loc[all_tokens_phono.id.isin(selected)][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'id','bert_token_id','utterance_order','transcript_id']]

,gloss,actual_phonology_no_dia,model_phonology_no_dia,id,bert_token_id,utterance_order,transcript_id
1032327,I want to read,,,16928243,1032327,310,42336
1032328,I want to read,ɑə,ɑə,16928243,1032328,310,42336
1032329,I want to read,wɑn,wɑnt,16928243,1032329,310,42336
1032330,I want to read,də,tu,16928243,1032330,310,42336
1032331,I want to read,wid,ɹid,16928243,1032331,310,42336
1032332,I want to read,,,16928243,1032332,310,42336
1067443,I want to read,,,16933485,1067443,631,42336
1067444,I want to read,ɑə,ɑə,16933485,1067444,631,42336
1067445,I want to read,wɑn,wɑnt,16933485,1067445,631,42336
1067446,I want to read,də,tu,16933485,1067446,631,42336


In [ ]:
### Find new test examples- successes

In [7]:
all_models = load_models.get_model_dict()

# CDL + Context +/- 20 is needed
# BERT + Context +/- 20 is needed
# Childes on train data.

# How to load properly with sample across models?
which_models = [
    'all/all/no_tags/20_context/childes',
    'all/all/no_tags/20_context/bert',
    'all/all/no_tags/0_context/data_unigram',
]

test_idx = 16928243

scores_across_models = examples_figure.get_scores_across_models(test_idx)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Change the initial tokenizer to be based on latest trained models, eventually.


NameError: name 'examples_figure' is not defined

In [ ]:
success_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_surprisal','token']]
success_example

words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
success_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_surprisal','token']]
success_example

words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
success_example = scores_across_models.loc[(scores_across_models.model == adult_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_surprisal','token']]
success_example

words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
success_example = scores_across_models.loc[(scores_across_models.model == unigram_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_surprisal','token']]
success_example

words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

## New extraction section

In [ ]:
yyy_utts_with_gloss = yyy_utts.copy() # Already merged with gloss previously
yyy_utts_with_gloss['num_tokens'] = [len(x.split(' ')) for x in yyy_utts_with_gloss['gloss']]
' | '.join(yyy_utts_with_gloss.loc[success_utts_with_gloss.num_tokens ==4 ].gloss)

In [ ]:
selected = yyy_utts_with_gloss.loc[yyy_utts_with_gloss.gloss=='you make your yyy'].utterance_id 

In [ ]:
all_tokens_phono.loc[all_tokens_phono.id.isin(selected)][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'id','bert_token_id','utterance_order','transcript_id']]

In [17]:
scores_across_models = examples_figure.get_scores_across_models(16813515)

NameError: name 'models' is not defined

In [ ]:
scores_across_models

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == childes_all_title)][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_surprisal','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == adult_all_title)][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_surprisal','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == unigram_title)][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_surprisal','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
 utt_glosses.loc[(utt_glosses.transcript_id == 42253) &
                (utt_glosses.utterance_order.isin(range(112-3, 112+3)))]